```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Intermediate Layer](#intermediate-layer)**
5. **[Ouput Layer](#output-layer)**


Originating story can be found here: [COR-681](https://logius-prd.jira.odc-noord.nl/browse/COR-681) and [COR-1056](https://logius-prd.jira.odc-noord.nl/browse/COR-1056)

# **FLOW DIAGRAMS**

---

`ADD DIAGRAM HERE!`

Required steps:

1. `ADD REQUIRMENT STEP LIST HERE!`

- **<font color=teal>IL</font>**: Intermediate Layer
- **<font color=teal>OL</font>**: Output Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/DataFactory/Statics/pl_ProcessVaccinationCampaignMasterData.ipynb",
        "src/DataFactory/Protocols/pl_ProcessVaccinationPlannedNl.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_NL]'))
CREATE TABLE [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] VARCHAR(50) NULL,
	[DATE_OF_STATISTICS] VARCHAR(50) NULL,
	[CAMPAIGN] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] VARCHAR(255) NULL
);
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL]'))
CREATE TABLE [VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] DATETIME NULL,
	[DATE_OF_STATISTICS] DATETIME NULL,
	[CAMPAIGN] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] INT NULL	-- ADD COLUMNS
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_RIVM_VACCINATION_CAMPAIGNS_NL]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL] (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [CAMPAIGN],
        [TOTAL_VACCINATED]
    )
    SELECT
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_REPORT]),
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_STATISTICS]),
        UPPER(TRIM([CAMPAIGN])),
        [TOTAL_VACCINATED]
    FROM
        [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_VACCINATION_CAMPAIGNS_NL])
END;
GO

# **OUTPUT LAYER**

---

## **<span style='color:teal'>RIVM VACCINATION CAMPAIGNS NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL]'))
CREATE TABLE [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] DATETIME NULL,
	[DATE_OF_STATISTICS] DATETIME NULL,
	[CAMPAIGN] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] INT NULL,
	[TOTAL_VACCINATED_7DAYS] INT NULL,
	[CUMSUM_TOTAL_VACCINATED] INT NULL,
);
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_VACCINATION_CAMPAIGNS_NL] 
	ON [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RIVM_VACCINATION_CAMPAIGNS_NL]
    ON [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] (
		[DATE_LAST_INSERTED]
	)
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_RIVM_REPEATING_SHOTS_ADMINISTERED_N]
AS
BEGIN
WITH CTE AS (
        SELECT 
			[ID],
            [DATE_OF_REPORT],
			[DATE_OF_STATISTICS],
            [CAMPAIGN],
            [TOTAL_VACCINATED]
        FROM [VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_VACCINATION_CAMPAIGNS_NL])
    ),
	CTE_7DAYS AS (
		SELECT 
			r.Id, 
			v.total_vaccinated
        FROM CTE r
		LEFT JOIN CTE v 
			ON r.CAMPAIGN = v.CAMPAIGN 
			AND v.DATE_OF_STATISTICS > dateadd(day, -7,  [dbo].[WEEK_START_ISO](r.[DATE_OF_STATISTICS])) --Week before most recent sunday
			AND v.DATE_OF_STATISTICS <= [dbo].[WEEK_START_ISO](r.[DATE_OF_STATISTICS]) --Most recent sunday
	),
	CTE_7DAYS_SUM AS (
		SELECT Id AS Id, sum(total_vaccinated) AS SUM_VACCINATED
		FROM CTE_7DAYS
		group by Id
	),
	CTE_ALLDAYS AS (
		SELECT 
			r.Id, 
			v.total_vaccinated
        FROM CTE r
		LEFT JOIN CTE v 
			ON r.CAMPAIGN = v.CAMPAIGN 
			AND v.DATE_OF_STATISTICS <= [dbo].[WEEK_START_ISO](r.[DATE_OF_STATISTICS]) --Most recent sunday
	),
	CTE_ALLDAYS_SUM AS (
		SELECT Id AS Id, sum(total_vaccinated) AS SUM_VACCINATED
		FROM CTE_ALLDAYS
		group by Id
	)
    INSERT INTO [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] (
            [DATE_OF_REPORT],
			[DATE_OF_STATISTICS],
            [CAMPAIGN],
            [TOTAL_VACCINATED],
            [TOTAL_VACCINATED_7DAYS],
            [CUMSUM_TOTAL_VACCINATED]
    )
    SELECT
        c.[DATE_OF_REPORT],
        c.[DATE_OF_STATISTICS],
		c.[CAMPAIGN],
        c.[TOTAL_VACCINATED],
		c7.SUM_VACCINATED AS TOTAL_VACCINATED_7DAYS,
		ca.SUM_VACCINATED AS CUMSUM_TOTAL_VACCINATED
    FROM CTE c
	INNER JOIN CTE_7DAYS_SUM c7 ON c.ID = c7.Id
	INNER JOIN CTE_ALLDAYS_SUM ca ON c.Id = ca.Id
	

END;
GO

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATION_CAMPAIGNS_NL] AS
    WITH CTE AS (
        SELECT
            [DATE_LAST_INSERTED],
            [DATE_OF_REPORT],
            [DATE_OF_STATISTICS],
            [CAMPAIGN],
            [TOTAL_VACCINATED],
            [CUMSUM_TOTAL_VACCINATED],
            [TOTAL_VACCINATED_7DAYS]
        FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] 
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINATION_CAMPAIGNS_NL] ) 
    )
    SELECT
            r.[ORDER] AS [VACCINE_CAMPAIGN_ORDER],
            r.[CAMPAIGN_LABEL_NL] AS [VACCINE_CAMPAIGN_NAME_NL],
            r.[CAMPAIGN_LABEL_EN] AS [VACCINE_CAMPAIGN_NAME_EN],
            c.[CUMSUM_TOTAL_VACCINATED] AS [VACCINE_ADMINISTERED_TOTAL],
            c.[TOTAL_VACCINATED_7DAYS] AS [VACCINE_ADMINISTERED_LAST_WEEK],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([dbo].[WEEK_START_ISO](c.[DATE_OF_STATISTICS])) AS [DATE_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX]([dbo].[WEEK_START_ISO](c.[DATE_OF_STATISTICS])) AS [DATE_END_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX](dateadd(day, -6,  [dbo].[WEEK_START_ISO](c.[DATE_OF_STATISTICS]))) AS [DATE_START_UNIX],
            [dbo].[CONVERT_DATETIME_TO_UNIX](c.[DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
    FROM CTE c
    INNER JOIN (
        SELECT
            [CAMPAIGN_CODE],
            [CAMPAIGN_LABEL_NL],
            [CAMPAIGN_LABEL_EN],
            [ORDER]
        FROM [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA] 
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_VACCINATION_CAMPAIGNS_MASTER_DATA]) 
    ) r ON c.CAMPAIGN = r.[CAMPAIGN_CODE]
    WHERE c.[DATE_OF_STATISTICS] = (SELECT MAX([DATE_OF_STATISTICS]) FROM  CTE)
GO

In [ ]:
CREATE OR ALTER VIEW VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED  
AS

 SELECT CAST(VACCINE_CAMPAIGN_ORDER as INT) AS VACCINE_CAMPAIGN_ORDER,
        CAST(DATE_OF_INSERTION_UNIX as BIGINT) as DATE_OF_INSERTION_UNIX,
        VACCINE_CAMPAIGN_NAME_NL,
        VACCINE_CAMPAIGN_NAME_EN,
        CAST(VACCINE_ADMINISTERED_TOTAL as INT) as VACCINE_ADMINISTERED_TOTAL,
        CAST(VACCINE_ADMINISTERED_LAST_WEEK as INT) as VACCINE_ADMINISTERED_LAST_WEEK,
        CAST(DATE_UNIX as BIGINT) as DATE_UNIX,
        CAST(DATE_START_UNIX as BIGINT) as DATE_START_UNIX,
        CAST(DATE_END_UNIX as BIGINT) as DATE_END_UNIX
 FROM 
 (
 	-- Get the rows for the different campaigns 
 	SELECT A.[Key], 
            CAST(A.[Group] as INT) as VACCINE_CAMPAIGN_ORDER, 
            A.Value, 
            dbo.CONVERT_DATETIME_TO_UNIX(A.Date_Last_Inserted) AS DATE_OF_INSERTION_UNIX
 	FROM VWSStage.RIVM_VACCINATION_PLANNED_NL A
 	WHERE LOWER(Category) = 'vaccine_Campaigns.vaccine_Campaigns'
         AND DATE_Last_Inserted=(SELECT MAX(Date_Last_Inserted) FROM VWSStage.RIVM_VACCINATION_PLANNED_NL)
 	
 	UNION
 	
 	SELECT A.[Key], 
            B.[GROUP] as VACCINE_CAMPAIGN_ORDER, 
            CAST(dbo.CONVERT_DATETIME_TO_UNIX(dbo.TRY_CONVERT_TO_DATETIME(A.Value)) as nvarchar(max))as [Value],
            dbo.CONVERT_DATETIME_TO_UNIX(A.Date_Last_Inserted) AS DATE_OF_INSERTION_UNIX
 	FROM VWSStage.RIVM_VACCINATION_PLANNED_NL A
 	CROSS JOIN (
                     SELECT Distinct [GROUP] 
                     FROM VWSStage.RIVM_VACCINATION_PLANNED_NL
                     WHERE NOT (([GROUP] is NULL) OR (UPPER([GROUP])='NULL'))
                         AND DATE_Last_Inserted=(SELECT MAX(Date_Last_Inserted) FROM VWSStage.RIVM_VACCINATION_PLANNED_NL)
                 ) B
 	WHERE UPPER(A.Category) = 'VACCINE_CAMPAIGNS'
 ) t
 -- Now pivot the table per (implicit) Group / VACCINE_CAMPAIGN_ORDER, the rows become columns now
 PIVOT (
 	Min([Value])
 	FOR [Key] IN ([VACCINE_CAMPAIGN_NAME_NL]
 	            , [VACCINE_CAMPAIGN_NAME_EN]
 				, [VACCINE_ADMINISTERED_TOTAL]
 				, [VACCINE_ADMINISTERED_LAST_WEEK]
 				, [DATE_UNIX]
                 ,[DATE_START_UNIX]
 				, [DATE_END_UNIX])
 ) AS Pt

GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- View aanmaken in DataTino_Protos_1.Views
DECLARE @ViewID INT = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED%')
DELETE FROM DATATINO_PROTO_1.CONFIGURATIONS WHERE View_ID = @ViewID
DELETE FROM Datatino_Proto_1.Views WHERE Name Like '%VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED%'

INSERT INTO Datatino_Proto_1.Views (Last_Update_name, Constraint_Key_name, Constraint_Value, Grouped_Key_Name, Grouped_Last_Update_Name, Name, Description)
SELECT 'DATE_UNIX',
       NULL,
       NULL,
       NULL,
       NULL,
       'VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED',
       'VIEW: VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED FOR vaccine_campaigns/vaccine_campaigns'
       

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINATION_CAMPAIGNS_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_campaigns_disabled',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'DATE_UNIX|DATE_END_UNIX|DATE_START_UNIX|DATE_OF_INSERTION_UNIX',
        @layout_type_id INT = 3,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#') 
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_RIVM_VACCINATION_CAMPAIGNS_NL',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vaccine_campaigns_disabled/vaccine_campaigns_disabled',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'VACCINE_CAMPAIGN_ORDER|VACCINE_CAMPAIGN_NAME_NL|VACCINE_CAMPAIGN_NAME_EN|VACCINE_ADMINISTERED_TOTAL|VACCINE_ADMINISTERED_LAST_WEEK',
        @layout_type_id INT = 4,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- Naam van bestaande configuratie aanpassen naar _disabled en Active naar 0
DECLARE  @ViewID INT = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSDEST.V_RIVM_VACCINATION_CAMPAIGNS_NL%')
delete from  [DATATINO_PROTO_1].[CONFIGURATIONS] 
WHERE VIEW_ID = @ViewID


-- Twee regels toevoegen aan Datatino_Proto_1.Configurations
SET @ViewID = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED%')
DECLARE @ProtoID INT = (SELECT ID FROM Datatino_Proto_1.Protos WHERE Name = 'NL')

-- Eerst evt al aanwezige regels verwijderen
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE Proto_ID = @ProtoID and View_ID = @ViewID

-- Regel toevoegen voor 'header'
INSERT INTO [DATATINO_PROTO_1].[CONFIGURATIONS] (Proto_ID, View_ID, Constrained, Grouped, Columns, Mapping, Layout_type_id, Mock_id, Active, Name, Description)
SELECT @ProtoID, 
       @ViewID, 
       0,
       0, 
       'DATE_UNIX|DATE_END_UNIX|DATE_START_UNIX|DATE_OF_INSERTION_UNIX',
       '=LOWER()',
       3,
       NULL,
       1,
       'vaccine_campaigns',
       'VIEW CONFIGURATION: VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED FOR vaccine_campaigns'

-- Regel toevoegen voor 'details'
INSERT INTO [DATATINO_PROTO_1].[CONFIGURATIONS] (Proto_ID, View_ID, Constrained, Grouped, Columns, Mapping, Layout_type_id, Mock_id, Active, Name, Description)
SELECT @ProtoID, 
       @ViewID, 
       0,
       0,
       'VACCINE_CAMPAIGN_ORDER|VACCINE_CAMPAIGN_NAME_NL|VACCINE_CAMPAIGN_NAME_EN|VACCINE_ADMINISTERED_TOTAL|VACCINE_ADMINISTERED_LAST_WEEK',
       '=LOWER()',
       4,
       NULL,
       1,
       'vaccine_campaigns/vaccine_campaigns',
       'VIEW CONFIGURATION: VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED FOR vaccine_campaigns/vaccine_campaigns'
       


